# Day 7: No Space Left On Device

In [406]:
open System.Text.RegularExpressions

let extract pattern content =
    Regex.Matches(content, pattern)
    |> Seq.map (fun m -> m.Value)

let (|Match|_|) (pattern: string) (value: string) = 
    if (Regex.IsMatch(value, pattern)) then
        Some (extract pattern value)
    else 
        None

let (|Prefix|_|) (pattern: string) (value: string) = 
    if (value.StartsWith(pattern)) then 
        Some (value.Remove(0, pattern.Length)) 
    else 
        None
        
let applyRight f (first, second) =
    (first, f second)
    
let countByOccurance c =
    Seq.filter (fun x -> x = c) >> Seq.length

In [407]:
let getCurrentDirectory (current:string) command =
    match command with
    | "$ cd /" -> "/"
    | "$ cd .." -> Regex.Replace(current, "/\w+?/$", "/")
    | Prefix "$ cd " directory -> current + directory + "/"
    | _ -> current
    
let addFiles directory fileSystem file =
    match file with
    | Match "^\d+" _ -> (directory, file) :: fileSystem
    | Prefix "dir " folder -> (directory, directory + folder + "/") :: fileSystem
    | _ -> fileSystem

In [408]:
let rec createFileSystem currentDirectory fileSystem commands =
    match commands with
    | [] -> fileSystem
    | head::tail -> 
        let directory = getCurrentDirectory currentDirectory head
        let update = addFiles directory fileSystem head
        createFileSystem directory update tail

let findSize fileSystem (content: string) =
    match content with
    | Match "^\d+" matches -> Seq.head matches |> int
    | _ -> (fileSystem |> Seq.find (fun (key, value) ->  key = content) |> snd )
    
let sizeOf (sizeReport) (directory, contents) =
    let size = contents |> Seq.map (findSize sizeReport) |> Seq.sum
    (directory, size) :: sizeReport

let fs =
    File.ReadAllLines("Day7.txt")
    |> Seq.toList
    |> createFileSystem "" []
    |> Seq.groupBy fst
    |> Seq.map (applyRight (Seq.map snd))
    |> Seq.sortByDescending (fst >> countByOccurance '/')
    |> Seq.fold sizeOf List.Empty
    

Find all of the directories with a total size of at most 100000. What is the sum of the total sizes of those directories?

In [409]:
fs
|> Seq.filter (snd >> (>=) 100000)
|> Seq.map snd
|> Seq.sum

1908462

Find the smallest directory that, if deleted, would free up enough space on the filesystem to run the update. What is the total size of that directory?

In [410]:
let total = 70000000
let required = 30000000
let used = fs |> Seq.find (fst >> (=) "/") |> snd
let freeSpace = total - used
let deletionSize = required - freeSpace

fs
|> Seq.filter (snd >> (<=) deletionSize)
|> Seq.sortBy snd
|> Seq.head
|> snd

3979145